## Carvana Image Masking: Image Segmentation with LoRA

Dataset Source: https://www.kaggle.com/datasets/ipythonx/carvana-image-masking-png

#### Import Necessary Libraries

In [21]:
import os, sys, glob, shutil
os.environ['TOKENIZERS_PARALLELISM']='false'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset

import transformers
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import AutoImageProcessor, AutoModelForSemanticSegmentation

import torchvision

import evaluate

from PIL import Image

import peft
from peft import LoraConfig, get_peft_model

!git lfs install

Git LFS initialized.


#### Display Library Versions

In [22]:
print("Python :".rjust(18), sys.version[0:6])
print("NumPy :".rjust(18), np.__version__)
print("Pandas :".rjust(18), pd.__version__)
print("Torch :".rjust(18), torch.__version__)
print("Torch Vision :".rjust(18), torchvision.__version__)
print("Transformers :".rjust(18), transformers.__version__)
print("Evaluate :".rjust(18), evaluate.__version__)
print("PEFT :".rjust(18), peft.__version__)

          Python : 3.9.12
           NumPy : 1.25.1
          Pandas : 2.0.3
           Torch : 2.0.1
    Torch Vision : 0.15.2
    Transformers : 4.29.1
        Evaluate : 0.4.0
            PEFT : 0.3.0


#### Basic Values/Constants

In [23]:
MODEL_CKPT = "nvidia/mit-b0"
MODEL_NAME = f'{MODEL_CKPT.split("/")[-1]}-Image_segmentation-Carvana_Image_Masking'

LR = 5e-4
NUM_OF_EPOCHS = 10

STRATEGY = "steps"
REPORTS_TO = "tensorboard"

#### Create Dataset Class

In [24]:
class ImageSegmentationDataset(Dataset):
    """
    Class to create an Image 
    (Semantic) Segmentation dataset
    """

    def __init__(self, parent_dir, image_processor, train=True):
        """
        Args:
            parent_dir (string): Root directory of the dataset containing the images + annotations.
            image_processor: image processor used to prepare images + segmentation maps.
            train (bool): Whether to load "training" or "validation" images + annotations.
        """
        self.parent_dir = parent_dir
        self.image_processor = image_processor
        self.train = train

        sub_path = "training" if self.train else "testing"
        self.img_dir = os.path.join(self.parent_dir, sub_path, "images")
        self.ann_dir = os.path.join(self.parent_dir, sub_path, "annotations")
        
        # read images
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
            image_file_names.extend(files)
        self.images = sorted(image_file_names)
        
        # read annotations
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
            annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

        assert len(self.images) == len(self.annotations), \
            "There must be as many images as there are segmentation maps"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = Image.open(os.path.join(self.img_dir, self.images[idx]))
        segmentation_map = Image.open(os.path.join(self.ann_dir, self.annotations[idx]))

        # randomly crop + pad both image and segmentation map to same size
        encoded_inputs = self.image_processor(image, segmentation_map, return_tensors="pt")

        for k,v in encoded_inputs.items():
            encoded_inputs[k].squeeze_() # remove batch dimension

        return encoded_inputs

#### Define Image Processor & Ingest Training & Testing Datasets

In [25]:
image_processor = AutoImageProcessor.from_pretrained(MODEL_CKPT, 
                                                     do_reduce_labels=False)

training_data_folder = "/Users/briandunn/Desktop/Image Segmentation Projects/Carvana Image Masking/data"

train_ds = ImageSegmentationDataset(parent_dir=training_data_folder, 
                                      image_processor=image_processor)

testing_data_folder = "/Users/briandunn/Desktop/Image Segmentation Projects/Carvana Image Masking/data"

test_ds = ImageSegmentationDataset(parent_dir=testing_data_folder, 
                                      image_processor=image_processor, 
                                      train=False)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/Users/briandunn/Desktop/python_venvs/torch_cv/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [26]:
print("Number of training examples:", len(train_ds))
print("Number of validation examples:", len(test_ds))

Number of training examples: 4070
Number of validation examples: 1018


#### Shape of Sample (Pixel Values Feature)

In [27]:
sample = train_ds[12]

sample['pixel_values'].shape

torch.Size([3, 512, 512])

#### Shape of Sample (Labels Feature)

In [28]:
sample['labels'].shape

torch.Size([512, 512])

#### Sample Tensors Output

In [29]:
sample['pixel_values']

tensor([[[2.0434, 2.0434, 2.0434,  ..., 2.0605, 2.0605, 2.0605],
         [2.0434, 2.0434, 2.0434,  ..., 2.0605, 2.0434, 2.0605],
         [2.0434, 2.0434, 2.0434,  ..., 2.0434, 2.0434, 2.0434],
         ...,
         [0.9817, 0.9817, 0.9646,  ..., 0.9474, 0.9474, 0.9474],
         [0.9646, 0.9646, 0.9646,  ..., 0.9303, 0.9474, 0.9474],
         [0.9817, 0.9817, 0.9646,  ..., 0.9474, 0.9303, 0.9303]],

        [[2.2185, 2.2185, 2.2185,  ..., 2.2360, 2.2360, 2.2360],
         [2.2185, 2.2185, 2.2185,  ..., 2.2360, 2.2185, 2.2360],
         [2.2185, 2.2185, 2.2185,  ..., 2.2185, 2.2185, 2.2185],
         ...,
         [1.2031, 1.2031, 1.1856,  ..., 1.1331, 1.1331, 1.1331],
         [1.1856, 1.1856, 1.1856,  ..., 1.1155, 1.1331, 1.1331],
         [1.2031, 1.2031, 1.1856,  ..., 1.1331, 1.1155, 1.1155]],

        [[2.3960, 2.3960, 2.3960,  ..., 2.4483, 2.4483, 2.4483],
         [2.3960, 2.3960, 2.3960,  ..., 2.4483, 2.4308, 2.4483],
         [2.3960, 2.3960, 2.3960,  ..., 2.4308, 2.4308, 2.

#### Unique Label Values

In [30]:
sample['labels'].squeeze().unique()

tensor([0, 1])

#### Create Conversions Between String & Integer Values For Label Values

In [31]:
id2label = {0: "background", 1: "vehicle"}
label2id = {label: idx for idx, label in id2label.items()}

unqiue_label_values = list(id2label.values())

NUM_OF_LABELS = len(unqiue_label_values)

print(f"List of Unique Label Values: {unqiue_label_values}")
print(f"Number of Unique Label Values: {NUM_OF_LABELS}")
print(f"label2id: {label2id}")
print(f"id2label: {id2label}")

List of Unique Label Values: ['background', 'vehicle']
Number of Unique Label Values: 2
label2id: {'background': 0, 'vehicle': 1}
id2label: {0: 'background', 1: 'vehicle'}


#### Define Compute Metrics Function

In [32]:
metric = evaluate.load("mean_iou")
    
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=NUM_OF_LABELS,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

#### Define Function to Display Number of Trainable Parameters

In [33]:
def print_trainable_parameters(model):
    """
    This function prints the following values:
    - the number of parameters
    - the trainable parameters 
    - the percentage of total parameters 
        that are trainable.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"| Trainable parameters: {trainable_params} |")
    print(f"| All parameters: {all_param} |")
    print(f"| % Trainable: {100 * trainable_params / all_param:.2f}")    

#### Define Base Model

In [34]:
model = AutoModelForSemanticSegmentation.from_pretrained(
    MODEL_CKPT, 
    id2label=id2label, 
    label2id=label2id,
    ignore_mismatched_sizes=True)

print_trainable_parameters(model)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.3.proj.bias', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.classifier.bias', 'decode_head.linear_c.0.

| Trainable parameters: 3714658 |
| All parameters: 3714658 |
| % Trainable: 100.00


#### Define LoRA Configuration

In [35]:
lora_config = LoraConfig(
    r=32, 
    lora_alpha=32,
    target_modules=['query', 'values'],
    lora_dropout=0.1,
    bias='lora_only',
    modules_to_save=['decode_head'],
)

#### Wrap Base Model With LoRA Configuration

In [36]:
lora_model = get_peft_model(model, 
                            lora_config)

print_trainable_parameters(lora_model)

| Trainable parameters: 857092 |
| All parameters: 4175460 |
| % Trainable: 20.53


#### Define Early Stopping Callback

In [37]:
early_stopping_callback = [EarlyStoppingCallback(early_stopping_patience=4)]

#### Define Training Arguments

In [38]:
args = TrainingArguments(
    MODEL_NAME,
    learning_rate=LR,
    remove_unused_columns=False,
    num_train_epochs=NUM_OF_EPOCHS,
    auto_find_batch_size=True,
    evaluation_strategy=STRATEGY,
    eval_steps=509,
    save_strategy=STRATEGY,
    save_steps=509,
    save_total_limit=6,
    label_names=['labels'],
    logging_first_step=True,
    logging_strategy=STRATEGY,
    logging_steps=51,
    load_best_model_at_end=True,
    report_to=REPORTS_TO,
    hub_private_repo=True,
    push_to_hub=True
)

#### Define Trainer

In [39]:
trainer = Trainer(
    lora_model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
    callbacks=early_stopping_callback,
)

Cloning https://huggingface.co/DunnBC22/mit-b0-Image_segmentation-Carvana_Image_Masking into local empty directory.


#### Train Model

In [40]:
trainer.train()

/Users/briandunn/Desktop/python_venvs/torch_cv/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5090 [00:00<?, ?it/s]

{'loss': 0.6576, 'learning_rate': 0.000499901768172888, 'epoch': 0.0}
{'loss': 0.115, 'learning_rate': 0.0004949901768172888, 'epoch': 0.1}
{'loss': 0.0314, 'learning_rate': 0.0004899803536345776, 'epoch': 0.2}
{'loss': 0.0219, 'learning_rate': 0.0004849705304518664, 'epoch': 0.3}
{'loss': 0.019, 'learning_rate': 0.00047996070726915517, 'epoch': 0.4}
{'loss': 0.0171, 'learning_rate': 0.000474950884086444, 'epoch': 0.5}
{'loss': 0.0155, 'learning_rate': 0.00046994106090373283, 'epoch': 0.6}
{'loss': 0.0149, 'learning_rate': 0.00046493123772102166, 'epoch': 0.7}
{'loss': 0.0149, 'learning_rate': 0.00045992141453831043, 'epoch': 0.8}
{'loss': 0.0137, 'learning_rate': 0.0004549115913555992, 'epoch': 0.9}


  0%|          | 0/128 [00:00<?, ?it/s]

Trainer is attempting to log a value of "[0.9946260327775679, 0.9799209877962288]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9968763581592542, 0.99146628287315]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


...

{'eval_loss': 0.007037504576146603, 'eval_mean_iou': 0.9916168620753708, 'eval_mean_accuracy': 0.9960535371714071, 'eval_overall_accuracy': 0.9972055731213163, 'eval_per_category_iou': [0.9964702556912872, 0.9867634684594544], 'eval_per_category_accuracy': [0.9980368172126497, 0.9940702571301645], 'eval_runtime': 264.4531, 'eval_samples_per_second': 3.849, 'eval_steps_per_second': 0.484, 'epoch': 9.0}
{'loss': 0.0079, 'learning_rate': 4.911591355599214e-05, 'epoch': 9.02}
{'loss': 0.0072, 'learning_rate': 4.410609037328094e-05, 'epoch': 9.12}
{'loss': 0.0076, 'learning_rate': 3.9096267190569745e-05, 'epoch': 9.22}
{'loss': 0.0087, 'learning_rate': 3.408644400785855e-05, 'epoch': 9.32}
{'loss': 0.0077, 'learning_rate': 2.9076620825147347e-05, 'epoch': 9.42}
{'loss': 0.0074, 'learning_rate': 2.406679764243615e-05, 'epoch': 9.52}
{'loss': 0.0074, 'learning_rate': 1.905697445972495e-05, 'epoch': 9.62}
{'loss': 0.0075, 'learning_rate': 1.4047151277013753e-05, 'epoch': 9.72}
{'loss': 0.0075,

  0%|          | 0/128 [00:00<?, ?it/s]

Trainer is attempting to log a value of "[0.9964996655500316, 0.9868763925617403]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9980006976075766, 0.994318466698934]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.00696456478908658, 'eval_mean_iou': 0.9916880290558859, 'eval_mean_accuracy': 0.9961595821532553, 'eval_overall_accuracy': 0.9972290383809208, 'eval_per_category_iou': [0.9964996655500316, 0.9868763925617403], 'eval_per_category_accuracy': [0.9980006976075766, 0.994318466698934], 'eval_runtime': 264.186, 'eval_samples_per_second': 3.853, 'eval_steps_per_second': 0.485, 'epoch': 10.0}
{'train_runtime': 29191.0045, 'train_samples_per_second': 1.394, 'train_steps_per_second': 0.174, 'train_loss': 0.010749788351049592, 'epoch': 10.0}


TrainOutput(global_step=5090, training_loss=0.010749788351049592, metrics={'train_runtime': 29191.0045, 'train_samples_per_second': 1.394, 'train_steps_per_second': 0.174, 'train_loss': 0.010749788351049592, 'epoch': 10.0})

#### Save Model

In [41]:
lora_model.save_pretrained(f'{MODEL_NAME}')

#### Push Model to Hub

In [42]:
trainer.push_to_hub("All DUNN!!!")

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/16.0M [00:00<?, ?B/s]

Upload file adapter_model.bin:   0%|          | 1.00/1.78M [00:00<?, ?B/s]

Upload file runs/Jul31_22-37-29_Brians-Mac-mini/events.out.tfevents.1690861054.Brians-Mac-mini.7948.2:   0%|  …

To https://huggingface.co/DunnBC22/mit-b0-Image_segmentation-Carvana_Image_Masking
   2159b20..0193be1  main -> main

To https://huggingface.co/DunnBC22/mit-b0-Image_segmentation-Carvana_Image_Masking
   0193be1..b70b818  main -> main



'https://huggingface.co/DunnBC22/mit-b0-Image_segmentation-Carvana_Image_Masking/commit/0193be1ddbd4f50fba521581127794c93c072792'

### Notes & Other Takeaways From This Project

****
- I am surprised at both how well the results are of this project and how quickly this project converged on the optimal settings. 

- Originally, I ran this project for 50 epochs (but due to unfortunate circumstances, it timed out before finishing), so I trained it for 10 epochs instead. The metrics for this model versus the 50 epoch model are not much different.

****

### Citations

- Model Checkpoint (nvidia/mit-b0)
    > @article{DBLP:journals/corr/abs-2105-15203, author = {Enze Xie and Wenhai Wang and Zhiding Yu and Anima Anandkumar and Jose M. Alvarez and Ping Luo}, title = {SegFormer: Simple and Efficient Design for Semantic Segmentation with Transformers}, journal = {CoRR}, volume = {abs/2105.15203}, year = {2021}, url = {https://arxiv.org/abs/2105.15203}, eprinttype = {arXiv}, eprint = {2105.15203}, timestamp = {Wed, 02 Jun 2021 11:46:42 +0200}, biburl = {https://dblp.org/rec/journals/corr/abs-2105-15203.bib}, bibsource = {dblp computer science bibliography, https://dblp.org}}

- Metrics (mean_iou)
    > @software{MMSegmentation_Contributors_OpenMMLab_Semantic_Segmentation_2020, author = {{MMSegmentation Contributors}}, license = {Apache-2.0}, month = {7}, title = {{OpenMMLab Semantic Segmentation Toolbox and Benchmark}}, url = {https://github.com/open-mmlab/mmsegmentation}, year = {2020}}